# Feature Engineering using featuretools



# Create an entity/entitysets

Automated feature engineering automatcially creates many candidate features out of dataset from which we select the best features for training. 

An entity is a single table or a dataframe in pandas.

An entity set is a collection of tables and the relationships between them.

In [6]:
import pandas as pd
import featuretools as ft
import os

# ignore warnings from pandas
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#Making different dataframes from csv
loans = pd.read_csv(os.getcwd()+ "\Data\cleaned_data.csv", low_memory=False)
borrowers = loans[['member_id','emp_length','home_ownership','annual_inc','addr_state','mths_since_last_delinq','tot_cur_bal']]
loangradedata = ['A','B','C','D','E','F','G']
loangrade = pd.DataFrame(loangradedata,columns=['grade'])

#create new entityset
es = ft.EntitySet(id = 'loandetails')

#create an entity from loan dataset
es = es.entity_from_dataframe(entity_id = 'loans', dataframe = loans, index = 'id' ,variable_types = {'loan_status': ft.variable_types.Categorical}) #,time_index = 'term'

es = es.entity_from_dataframe(entity_id = 'borrowers', dataframe = borrowers, index = 'member_id' ) #,time_index = 'term'

es = es.entity_from_dataframe(entity_id = 'loangrade', dataframe = loangrade, index = 'grade' ) #,time_index = 'term'

#Look at the entity
#es


# Feature Primitves

It is an operation implemented to create feature.Featuretools works on Deep Feature Synthesis, which means basically using feature primitives like transformation and aggregations stacked on top of each other to form new features.

Transformation : 
It is implemented on a single table by creating new features from  the existing list of columns

Aggregation:
Its is performed across many tables and we use a one-to-many relationship to group observations, further we calculate statistcs for these features

Create new features using specified primitives. This can be done by selecting the primitives from list mentioned below:
#primitives = ft.list_primitives()
#primitives[primitives['type'] == 'transform']
#print(primitives[primitives['type'] == 'transform'])
#print(primitives[primitives['type'] == 'aggregation'])

In [7]:
#Relationship between borrowers and loans
r_borrower_loans = ft.Relationship(es['borrowers']['member_id'],
                                    es['loans']['member_id'])
r_loans_grades = ft.Relationship(es['loangrade']['grade'],
                                es['loans']['grade'])

# Add the relationship to the entity set
es = es.add_relationship(r_borrower_loans)
es = es.add_relationship(r_loans_grades)

#es

#'percentile','cum_mean','cum_sum','cum_min', 'diff','absolute','year'
#max_depth=2

# features, feature_names = ft.dfs(entityset = es, target_entity = 'loans', 
#                                  agg_primitives = ['count','mean','min','max'],
#                                   trans_primitives = ['year','divide'],
#                                  max_depth = 2)

# Perform deep feature synthesis without specifying primitives
dpfeatures, dpfeature_names = ft.dfs(entityset=es, target_entity='loans', max_depth = 2)

dpfeatures.info()
dpfeatures.columns.values.tolist()
dpfeatures.to_csv(os.getcwd()+ "\Data\FT_data.csv",index = False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254190 entries, 54734 to 68604659
Columns: 298 entries, member_id to loangrade.MODE(loans.loan_status)
dtypes: float64(155), int64(108), object(35)
memory usage: 579.9+ MB


# Eliminating extra features

In [15]:
import pandas as pd
import featuretools as ft
import os

# ignore warnings from pandas
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

feautures = pd.read_csv(os.getcwd()+ "\Data\FT_data.csv", low_memory=False)
#creating copy of features
features_copy = feautures.copy()
#dpfeatures_copy.info()
#dpfeatures = dpfeatures_copy.copy()

print("Before elimination")
feautures.info()

feautures.drop(['borrowers.emp_length', 
                 'borrowers.home_ownership',
                 'borrowers.annual_inc',
                 'borrowers.addr_state',
                 'borrowers.mths_since_last_delinq',
                 'borrowers.tot_cur_bal',
                 'borrowers.MAX(loans.loan_amnt)',
                 'borrowers.MAX(loans.funded_amnt)',
                 'borrowers.MAX(loans.int_rate)',
                 'borrowers.MAX(loans.annual_inc)',
                 'borrowers.MAX(loans.dti)',
                 'borrowers.MAX(loans.delinq_2yrs)',
                 'borrowers.MAX(loans.mths_since_last_delinq)',
                 'borrowers.MAX(loans.open_acc)',
                 'borrowers.MAX(loans.pub_rec)',
                 'borrowers.MAX(loans.revol_bal)',
                 'borrowers.MAX(loans.total_pymnt)',
                 'borrowers.MAX(loans.recoveries)',
                 'borrowers.MAX(loans.annual_inc_joint)',
                 'borrowers.MAX(loans.dti_joint)',
                 'borrowers.MAX(loans.tot_cur_bal)',
                 'borrowers.MAX(loans.term(years))',
                 'borrowers.MAX(loans.total_rec_int)',
                 'borrowers.MAX(loans.total_rec_late_fee)',
                 'borrowers.MIN(loans.loan_amnt)',
                 'borrowers.MIN(loans.funded_amnt)',
                 'borrowers.MIN(loans.int_rate)',
                 'borrowers.MIN(loans.annual_inc)',
                 'borrowers.MIN(loans.dti)',
                 'borrowers.MIN(loans.delinq_2yrs)',
                 'borrowers.MIN(loans.mths_since_last_delinq)',
                 'borrowers.MIN(loans.open_acc)',
                 'borrowers.MIN(loans.pub_rec)',
                 'borrowers.MIN(loans.revol_bal)',
                 'borrowers.MIN(loans.total_pymnt)',
                 'borrowers.MIN(loans.recoveries)',
                 'borrowers.MIN(loans.annual_inc_joint)',
                 'borrowers.MIN(loans.dti_joint)',
                 'borrowers.MIN(loans.tot_cur_bal)',
                 'borrowers.MEAN(loans.loan_amnt)',
                 'borrowers.MIN(loans.term(years))',
                 'borrowers.MIN(loans.total_rec_int)',
                 'borrowers.MIN(loans.total_rec_late_fee)',
                 'borrowers.MEAN(loans.funded_amnt)',
                 'borrowers.MEAN(loans.int_rate)',
                 'borrowers.MEAN(loans.annual_inc)',
                 'borrowers.MEAN(loans.dti)',
                 'borrowers.MEAN(loans.delinq_2yrs)',
                 'borrowers.MEAN(loans.mths_since_last_delinq)',
                 'borrowers.MEAN(loans.open_acc)',
                 'borrowers.MEAN(loans.pub_rec)',
                 'borrowers.MEAN(loans.revol_bal)',
                 'borrowers.MEAN(loans.total_pymnt)',
                 'borrowers.MEAN(loans.recoveries)',
                 'borrowers.MEAN(loans.annual_inc_joint)',
                 'borrowers.MEAN(loans.dti_joint)',
                 'borrowers.MEAN(loans.tot_cur_bal)',
                 'borrowers.MEAN(loans.term(years))',
                 'borrowers.MEAN(loans.total_rec_int)',
                 'borrowers.MEAN(loans.total_rec_late_fee)',
                 'borrowers.COUNT(loans)',
                 'borrowers.NUM_UNIQUE(loans.grade)',
                 'borrowers.NUM_UNIQUE(loans.sub_grade)',
                 'borrowers.NUM_UNIQUE(loans.emp_length)',
                 'borrowers.NUM_UNIQUE(loans.home_ownership)',
                 'borrowers.NUM_UNIQUE(loans.verification_status)',
                 'borrowers.NUM_UNIQUE(loans.issue_d)',
                 'borrowers.NUM_UNIQUE(loans.purpose)',
                 'borrowers.NUM_UNIQUE(loans.addr_state)',
                 'borrowers.NUM_UNIQUE(loans.earliest_cr_line)',
                 'borrowers.NUM_UNIQUE(loans.application_type)',
                 'borrowers.NUM_UNIQUE(loans.loan_status)',
                 'borrowers.MODE(loans.grade)',
                 'borrowers.MODE(loans.sub_grade)',
                 'borrowers.MODE(loans.emp_length)',
                 'borrowers.MODE(loans.home_ownership)',
                 'borrowers.MODE(loans.verification_status)',
                 'borrowers.MODE(loans.issue_d)',
                 'borrowers.MODE(loans.purpose)',
                 'borrowers.MODE(loans.addr_state)',
                 'borrowers.MODE(loans.earliest_cr_line)',
                 'borrowers.MODE(loans.application_type)',
                 'borrowers.MODE(loans.loan_status)',
                 'borrowers.SKEW(loans.loan_amnt)',
                 'borrowers.SKEW(loans.funded_amnt)',
                 'borrowers.SKEW(loans.int_rate)',
                 'borrowers.SKEW(loans.annual_inc)',
                 'borrowers.SKEW(loans.dti)',
                 'borrowers.SKEW(loans.delinq_2yrs)',
                 'borrowers.SKEW(loans.mths_since_last_delinq)',
                 'borrowers.SKEW(loans.open_acc)',
                 'borrowers.SKEW(loans.pub_rec)',
                 'borrowers.SKEW(loans.revol_bal)',
                 'borrowers.SKEW(loans.total_pymnt)',
                 'borrowers.SKEW(loans.recoveries)',
                 'borrowers.SKEW(loans.annual_inc_joint)',
                 'borrowers.SKEW(loans.dti_joint)',
                 'borrowers.SKEW(loans.tot_cur_bal)',
                 'borrowers.SKEW(loans.term(years))',
                 'borrowers.SKEW(loans.total_rec_int)',
                 'borrowers.SKEW(loans.total_rec_late_fee)',
                 'borrowers.SUM(loans.loan_amnt)',
                 'borrowers.SUM(loans.funded_amnt)',
                 'borrowers.SUM(loans.int_rate)',
                 'borrowers.SUM(loans.annual_inc)',
                 'borrowers.SUM(loans.dti)',
                 'borrowers.SUM(loans.delinq_2yrs)',
                 'borrowers.SUM(loans.mths_since_last_delinq)',
                 'borrowers.SUM(loans.open_acc)',
                 'borrowers.SUM(loans.pub_rec)',
                 'borrowers.SUM(loans.revol_bal)',
                 'borrowers.SUM(loans.total_pymnt)',
                 'borrowers.SUM(loans.term(years))',
                 'borrowers.SUM(loans.recoveries)',
                 'borrowers.SUM(loans.annual_inc_joint)',
                 'borrowers.SUM(loans.total_rec_int)',
                 'borrowers.SUM(loans.dti_joint)',
                 'borrowers.SUM(loans.tot_cur_bal)',
                 'borrowers.STD(loans.loan_amnt)',
                 'borrowers.STD(loans.funded_amnt)',
                 'borrowers.STD(loans.int_rate)',
                 'borrowers.STD(loans.annual_inc)',
                 'borrowers.STD(loans.dti)',
                 'borrowers.STD(loans.delinq_2yrs)',
                 'borrowers.STD(loans.mths_since_last_delinq)',
                 'borrowers.STD(loans.open_acc)',
                 'borrowers.STD(loans.pub_rec)',
                 'borrowers.STD(loans.revol_bal)',
                 'borrowers.STD(loans.total_pymnt)',
                 'borrowers.STD(loans.recoveries)',
                 'borrowers.STD(loans.annual_inc_joint)',
                 'borrowers.STD(loans.dti_joint)',
                 'borrowers.STD(loans.tot_cur_bal)',
                 'borrowers.STD(loans.term(years))',
                 'borrowers.STD(loans.total_rec_int)',
                 'borrowers.STD(loans.total_rec_int)',
                 'borrowers.STD(loans.total_rec_late_fee)',
                 'borrowers.SUM(loans.total_rec_late_fee)',
                 'loangrade.SUM(loans.int_rate)',
                 'loangrade.SUM(loans.annual_inc_joint)',
                 'loangrade.SUM(loans.dti_joint)',
                 'loangrade.SUM(loans.tot_cur_bal)',
                 'loangrade.SUM(loans.delinq_2yrs)',
                 'loangrade.SUM(loans.funded_amnt)',
                 'loangrade.SUM(loans.mths_since_last_delinq)',
                 'loangrade.SUM(loans.open_acc)',
                 'loangrade.SUM(loans.pub_rec)',
                 'loangrade.SUM(loans.revol_bal)',
                 'loangrade.SUM(loans.term(years))',
                 'loangrade.SUM(loans.annual_inc)',
                 'loangrade.SUM(loans.dti)',
                 'loangrade.SUM(loans.total_rec_int)',
                 'loangrade.SUM(loans.total_pymnt)',
                 'loangrade.SUM(loans.total_rec_late_fee)',
                 'loangrade.STD(loans.revol_bal)',
                 'loangrade.STD(loans.tot_cur_bal)',
                 'loangrade.MAX(loans.funded_amnt)',
                 'loangrade.MAX(loans.mths_since_last_delinq)',
                 'loangrade.MAX(loans.open_acc)',
                 'loangrade.MAX(loans.pub_rec)',
                 'loangrade.MAX(loans.annual_inc_joint)',
                 'loangrade.MAX(loans.tot_cur_bal)',
                 'loangrade.MAX(loans.dti)',
                 'loangrade.MAX(loans.delinq_2yrs)',
                 'loangrade.MAX(loans.annual_inc)',
                 'loangrade.MAX(loans.revol_bal)',
                 'loangrade.MAX(loans.total_pymnt)',
                 'loangrade.MAX(loans.total_rec_int)',
                 'loangrade.MAX(loans.total_rec_late_fee)',
                 'loangrade.SKEW(loans.funded_amnt)',
                 'loangrade.SKEW(loans.annual_inc)',
                 'loangrade.SKEW(loans.mths_since_last_delinq)',
                 'loangrade.SKEW(loans.open_acc)',
                 'loangrade.SKEW(loans.pub_rec)',
                 'loangrade.SKEW(loans.revol_bal)',
                 'loangrade.SKEW(loans.total_pymnt)',
                 'loangrade.SKEW(loans.recoveries)',
                 'loangrade.SKEW(loans.annual_inc_joint)',
                 'loangrade.SKEW(loans.dti_joint)',
                 'loangrade.SKEW(loans.tot_cur_bal)',
                 'loangrade.SKEW(loans.loan_amnt)',
                 'loangrade.SKEW(loans.term(years))',
                 'loangrade.SKEW(loans.int_rate)',
                 'loangrade.SKEW(loans.dti)',
                 'loangrade.SKEW(loans.delinq_2yrs)',
                 'loangrade.SKEW(loans.total_rec_int)',
                 'loangrade.SKEW(loans.total_rec_late_fee)',
                 'loangrade.MIN(loans.funded_amnt)',
                 'loangrade.MIN(loans.annual_inc)',
                 'loangrade.MIN(loans.dti)',
                 'loangrade.MIN(loans.delinq_2yrs)',
                 'loangrade.MIN(loans.mths_since_last_delinq)',
                 'loangrade.MIN(loans.open_acc)',
                 'loangrade.MIN(loans.pub_rec)',
                 'loangrade.MIN(loans.revol_bal)',
                 'loangrade.MIN(loans.total_pymnt)',
                 'loangrade.MIN(loans.recoveries)',
                 'loangrade.MIN(loans.annual_inc_joint)',
                 'loangrade.MIN(loans.dti_joint)',
                 'loangrade.MIN(loans.tot_cur_bal)',
                 'loangrade.MIN(loans.term(years))',
                 'loangrade.MIN(loans.total_rec_int)',
                 'loangrade.MIN(loans.total_rec_late_fee)',
                 'loangrade.NUM_UNIQUE(loans.sub_grade)',
                 'loangrade.NUM_UNIQUE(loans.emp_length)',
                 'loangrade.NUM_UNIQUE(loans.home_ownership)',
                 'loangrade.NUM_UNIQUE(loans.verification_status)',
                 'loangrade.NUM_UNIQUE(loans.issue_d)',
                 'loangrade.NUM_UNIQUE(loans.purpose)',
                 'loangrade.NUM_UNIQUE(loans.addr_state)',
                 'loangrade.NUM_UNIQUE(loans.earliest_cr_line)',
                 'loangrade.NUM_UNIQUE(loans.application_type)',
                 'loangrade.NUM_UNIQUE(loans.loan_status)',
                 'loangrade.NUM_UNIQUE(loans.member_id)',
                 'loangrade.MODE(loans.member_id)',
                 'loangrade.MODE(loans.emp_length)',
                 'loangrade.MODE(loans.verification_status)',
                 'loangrade.MODE(loans.issue_d)',
                 'loangrade.MODE(loans.earliest_cr_line)',
                 'loangrade.MODE(loans.application_type)'],axis =1, inplace=True)

#Check the dataframe size after elimination
print("After elimination")
feautures.info()
feautures.columns.values.tolist()
feautures.head()


Before elimination
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254190 entries, 0 to 254189
Columns: 298 entries, member_id to loangrade.MODE(loans.loan_status)
dtypes: float64(155), int64(108), object(35)
memory usage: 577.9+ MB
After elimination
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254190 entries, 0 to 254189
Data columns (total 79 columns):
member_id                                       254190 non-null int64
loan_amnt                                       254190 non-null int64
funded_amnt                                     254190 non-null int64
term(years)                                     254190 non-null int64
int_rate                                        254190 non-null float64
grade                                           254190 non-null object
sub_grade                                       254190 non-null object
emp_length                                      244222 non-null object
home_ownership                                  254190 non-null object


,member_id,loan_amnt,funded_amnt,term(years),int_rate,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,mths_since_last_delinq,open_acc,pub_rec,revol_bal,total_pymnt,total_rec_int,total_rec_late_fee,recoveries,application_type,annual_inc_joint,dti_joint,tot_cur_bal,loan_status,loangrade.SUM(loans.loan_amnt),loangrade.SUM(loans.recoveries),loangrade.STD(loans.loan_amnt),loangrade.STD(loans.funded_amnt),loangrade.STD(loans.term(years)),loangrade.STD(loans.int_rate),loangrade.STD(loans.annual_inc),loangrade.STD(loans.dti),loangrade.STD(loans.delinq_2yrs),loangrade.STD(loans.mths_since_last_delinq),loangrade.STD(loans.open_acc),loangrade.STD(loans.pub_rec),loangrade.STD(loans.total_pymnt),loangrade.STD(loans.total_rec_int),loangrade.STD(loans.total_rec_late_fee),loangrade.STD(loans.recoveries),loangrade.STD(loans.annual_inc_joint),loangrade.STD(loans.dti_joint),loangrade.MAX(loans.loan_amnt),loangrade.MAX(loans.term(years)),loangrade.MAX(loans.int_rate),loangrade.MAX(loans.recoveries),loangrade.MAX(loans.dti_joint),loangrade.MIN(loans.loan_amnt),loangrade.MIN(loans.int_rate),loangrade.MEAN(loans.loan_amnt),loangrade.MEAN(loans.funded_amnt),loangrade.MEAN(loans.term(years)),loangrade.MEAN(loans.int_rate),loangrade.MEAN(loans.annual_inc),loangrade.MEAN(loans.dti),loangrade.MEAN(loans.delinq_2yrs),loangrade.MEAN(loans.mths_since_last_delinq),loangrade.MEAN(loans.open_acc),loangrade.MEAN(loans.pub_rec),loangrade.MEAN(loans.revol_bal),loangrade.MEAN(loans.total_pymnt),loangrade.MEAN(loans.total_rec_int),loangrade.MEAN(loans.total_rec_late_fee),loangrade.MEAN(loans.recoveries),loangrade.MEAN(loans.annual_inc_joint),loangrade.MEAN(loans.dti_joint),loangrade.MEAN(loans.tot_cur_bal),loangrade.COUNT(loans),loangrade.MODE(loans.sub_grade),loangrade.MODE(loans.home_ownership),loangrade.MODE(loans.purpose),loangrade.MODE(loans.addr_state),loangrade.MODE(loans.loan_status)
0,80364,25000,25000,3,11.89,B,B4,6 months,RENT,85000.0,Verified,Aug-09,debt_consolidation,CA,19.48,0,Feb-94,22,10,0,28854,29324.32,4324.32,0.00,0.00,INDIVIDUAL,0,0.0,0,Fully Paid,945894400,6.211480e+06,7079.658799,7051.618869,0.571100,1.329668,58247.006723,7.490574,0.687317,21.721893,4.675371,0.411663,8086.754199,1250.252939,4.290512,508.464313,0.0,0.0,35000,5,14.09,29623.35,0.0,500,6.0,12403.057839,12365.174134,3.179117,11.549459,71218.260186,16.127525,0.225719,35.718789,10.770872,0.135256,14972.795563,12894.639797,1543.239956,0.511859,81.448141,0.0,0.0,101440.446389,76263,B3,MORTGAGE,debt_consolidation,CA,Fully Paid
1,114426,7000,7000,3,10.71,B,B5,6 months,RENT,65000.0,Not Verified,May-08,credit_card,NY,14.29,0,Oct-00,19,7,0,33623,8215.45,1215.45,0.00,0.00,INDIVIDUAL,0,0.0,0,Fully Paid,945894400,6.211480e+06,7079.658799,7051.618869,0.571100,1.329668,58247.006723,7.490574,0.687317,21.721893,4.675371,0.411663,8086.754199,1250.252939,4.290512,508.464313,0.0,0.0,35000,5,14.09,29623.35,0.0,500,6.0,12403.057839,12365.174134,3.179117,11.549459,71218.260186,16.127525,0.225719,35.718789,10.770872,0.135256,14972.795563,12894.639797,1543.239956,0.511859,81.448141,0.0,0.0,101440.446389,76263,B3,MORTGAGE,debt_consolidation,CA,Fully Paid
2,137225,25000,25000,3,16.99,D,D3,1 year,RENT,70000.0,Verified,Aug-14,debt_consolidation,NY,10.50,0,Jun-00,41,10,0,19878,6073.10,1038.95,44.56,3354.94,INDIVIDUAL,0,0.0,34561,Charged Off,580511400,9.492090e+06,8678.519636,8663.492716,0.942143,1.402420,44889.626741,8.015647,0.847737,21.688084,5.022941,0.493871,10398.527034,2252.920920,6.448289,857.071339,0.0,0.0,35000,5,20.31,25000.29,0.0,1000,6.0,14221.946200,14196.990298,3.664756,17.576956,69293.445628,17.732113,0.318340,34.524132,10.964697,0.179651,14919.985399,13778.694538,2422.734004,0.998920,232.546682,0.0,0.0,96792.478539,40818,D1,RENT,debt_consolidation,CA,Fully Paid
3,138150,1200,1200,3,13.11,C,C2,11 years,OWN,54000.0,Not Verified,Mar-10,debt_consolidation,TX,5.47,0,Jan-85,64,5,0,2584,1457.31,257.31,0.00,0.00,INDIVI

In [16]:
#Renaming the columns
feautures.rename(index = str, columns = {"loangrade.SUM(loans.loan_amnt)":"sum_loan_amnt",
                                        "loangrade.SUM(loans.recoveries)":"sum_recoveries",
                                        "loangrade.STD(loans.loan_amnt)":"std_loan_amnt",
                                        "loangrade.STD(loans.funded_amnt)":"std_funded_amnt",
                                        "loangrade.STD(loans.term(years))":"std_term",
                                        "loangrade.STD(loans.int_rate)":"std_int_rate",
                                        "loangrade.STD(loans.annual_inc)":"std_annual_inc",
                                        "loangrade.STD(loans.dti)":"std_dti",
                                        "loangrade.STD(loans.delinq_2yrs)":"std_delinq_2yrs",
                                        "loangrade.STD(loans.mths_since_last_delinq)":"std_mths_since_last_delinq",
                                        "loangrade.STD(loans.open_acc)":"std_open_cc",
                                        "loangrade.STD(loans.pub_rec)":"std_pub_rec",
                                        "loangrade.STD(loans.total_pymnt)":"std_total_paymnt",
                                        "loangrade.STD(loans.total_rec_int)":"std_total_rec_int",
                                        "loangrade.STD(loans.total_rec_late_fee)":"std_rec_late_fee",
                                        "loangrade.STD(loans.recoveries)":"std_recoveries",
                                        "loangrade.STD(loans.annual_inc_joint)":"std_annual_inc_joint",
                                        "loangrade.STD(loans.dti_joint)":"std_dti_joint",
                                        "loangrade.MAX(loans.loan_amnt)":"max_loan_amnt",
                                        "loangrade.MAX(loans.term(years))":"max_term",
                                        "loangrade.MAX(loans.int_rate)":"max_int_rate",
                                        "loangrade.MAX(loans.recoveries)":"max_recoveries",
                                        "loangrade.MAX(loans.dti_joint)":"max_dti_joint",
                                        "loangrade.MIN(loans.loan_amnt)":"min_loan_amt",
                                        "loangrade.MIN(loans.int_rate)":"min_int_rate",
                                        "loangrade.MEAN(loans.loan_amnt)":"mean_loan_amt",
                                        "loangrade.MEAN(loans.funded_amnt)":"mean_funded_amnt",
                                        "loangrade.MEAN(loans.term(years))":"mean_term",
                                        "loangrade.MEAN(loans.int_rate)":"mean_int_rate",
                                        "loangrade.MEAN(loans.annual_inc)":"mean_annual_inc",
                                        "loangrade.MEAN(loans.dti)":"mean_dti",
                                        "loangrade.MEAN(loans.delinq_2yrs)":"mean_delinq_2yrs",
                                        "loangrade.MEAN(loans.mths_since_last_delinq)":"mean_mths_since_last_delinq",
                                        "loangrade.MEAN(loans.open_acc)":"mean_open_cc",
                                        "loangrade.MEAN(loans.pub_rec)":"mean_pub_rec",
                                        "loangrade.MEAN(loans.total_pymnt)":"mean_total_paymnt",
                                        "loangrade.MEAN(loans.total_rec_int)":"mean_rec_int",
                                        "loangrade.MEAN(loans.recoveries)":"mean_recoveries",
                                        "loangrade.MEAN(loans.annual_inc_joint)":"mean_annual_inc_joint",
                                        "loangrade.MEAN(loans.dti_joint)":"mean_dti_joint",
                                        "loangrade.MEAN(loans.revol_bal)":"mean_revol_bal",
                                        "loangrade.MEAN(loans.tot_cur_bal)":"mean_total_cur_bal",
                                        "loangrade.MEAN(loans.total_rec_late_fee)":"mean_total_rec_late_fee", 
                                        "loangrade.COUNT(loans)":"loan_count",
                                        "loangrade.MODE(loans.sub_grade)":"mode_loan_subgrade",
                                        "loangrade.MODE(loans.home_ownership)":"mode_home_ownership",
                                        "loangrade.MODE(loans.purpose)":"mode_loan_purpose",
                                        "loangrade.MODE(loans.addr_state)":"mode_addr_state",
                                        "loangrade.MODE(loans.loan_status)":"mode_loan_status"}, inplace = True)

#Writing the data to csv
feautures.to_csv(os.getcwd()+ "\Data\FT_data_final.csv",index = False)